Practice this question here - https://platform.stratascratch.com/coding?is_freemium=1&difficulties=3&code_type=2

## Consecutive Days `Netflix`

Find all the users who were active for 3 consecutive days or more.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
sf_events.head()

def is_cons(user):
    temp = sf_events[sf_events['user_id'] == user]
    temp.sort_values(by='record_date',inplace=True)
    temp['prev'] = temp['record_date'].shift(1)
    temp['prev'] = (temp['record_date'] - temp['prev']).dt.days
    count = 0
    for i in temp['prev']:
        if i == 1:
            count += 1
            if count == 2:
                return True
        else:
            count = 0
    return False

sf_events[sf_events['user_id'].apply(is_cons)]['user_id'].unique()

## Best Selling Item `Ebay`

Find the best-selling item for each month (no need to separate months by year).

The best-selling item is determined by the highest total sales amount, calculated as: total_paid = unitprice * quantity. Output the month, description of the item, and the total amount paid.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
online_retail['month_no'] = online_retail['invoicedate'].dt.month
online_retail['total'] = online_retail['quantity'] * online_retail['unitprice']

temp = online_retail.groupby(['month_no','stockcode'],as_index=False)['total'].sum()

temp.sort_values(by=['month_no','total'],ascending=[True,False]).drop_duplicates(subset='month_no').merge(online_retail,on='stockcode')[['month_no_x','description','total_x']].rename(columns={
    'month_no_x' : 'month',
    'total_x' : 'total_paid'
}).drop_duplicates()

## Rank Variance Per Country `Meta`

Compare the total number of comments made by users in each country between December 2019 and January 2020. For each month, rank countries by total comments using dense ranking (i.e., avoid gaps between ranks) in descending order. Then, return the names of the countries whose rank improved from December to January.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
fb_comments_count['months'] = fb_comments_count['created_at'].dt.month

jan = fb_comments_count.query('months == 1')

dec = fb_comments_count.query('months == 12')

dec = dec.merge(fb_active_users,on='user_id').groupby('country',as_index=False)['number_of_comments'].sum()

dec['rank'] = dec['number_of_comments'].rank(ascending=False,method='dense')

jan = jan.merge(fb_active_users,on='user_id').groupby('country',as_index=False)['number_of_comments'].sum()

jan['rank'] = jan['number_of_comments'].rank(ascending=False,method='dense')

dec.merge(jan,on='country').query('rank_y < rank_x')['country']

## Actor Rating Difference Analysis `Google`

You are given a dataset of actors and the films they have been involved in, including each film's release date and rating. For each actor, calculate the difference between the rating of their most recent film and their average rating across all previous films (the average rating excludes the most recent one).


Return a list of actors along with their average lifetime rating, the rating of their most recent film, and the difference between the two ratings. If an actor has only one film, return 0 for the difference and their only film’s rating for both the average and latest rating fields.

In [ ]:
# Import your libraries
import pandas as pd
import numpy as np

# Start writing code
latest = actor_rating_shift.sort_values(by=['actor_name','release_date'],ascending=[True,False]).drop_duplicates(subset='actor_name')


def islatest(x):
    name = x['actor_name']
    date = x['release_date']
    if len(actor_rating_shift[actor_rating_shift['actor_name'] == name].values) == 1:
        return True

    if (name in latest['actor_name'].values) & (date in latest['release_date'].values):
        return False
    else:
        return True

avg = actor_rating_shift[actor_rating_shift.apply(islatest,axis=1)].groupby('actor_name',as_index=False)['film_rating'].mean()

df = latest.merge(avg,on='actor_name',how='left')[['actor_name','film_rating_y','film_rating_x']].rename(columns={'film_rating_x' : 'latest_rating', 'film_rating_y' : 'avg_rating'})

df


df['rating_difference'] = df['latest_rating'] - df['avg_rating']

df.sort_values(by='actor_name')

## Player with Longest Streak `Google`

You are given a table of tennis players and their matches that they could either win (W) or lose (L). Find the longest streak of wins. A streak is a set of consecutive won matches of one player. The streak ends once a player loses their next match. Output the ID of the player or players and the length of the streak.

In [ ]:
# Import your libraries
import pandas as pd

def consq(px):
    df = players_results.query(f'player_id == {px}').sort_values(by='match_date')
    cons = 0
    temp_cons = 0
    for i in df['match_result'].values :
        if i == 'W':
            temp_cons += 1
        else:
            if cons < temp_cons:
                cons = temp_cons
            temp_cons = 0
    return cons

# Start writing code
df = pd.DataFrame(columns=['player_id','streak'])
for j,i in enumerate(players_results['player_id'].unique()):
    df.loc[j] = {'player_id' : i , 'streak' : consq(i)}

df[df['streak'] == df['streak'].max()]

## Find the genre of the person with the most number of oscar winnings `Netflix`

Find the genre of the person with the most number of oscar winnings.
If there are more than one person with the same number of oscar wins, return the first one in alphabetic order based on their name. Use the names as keys when joining the tables.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = oscar_nominees.query('winner == True')['nominee'].value_counts().reset_index()

name = df[df['nominee'] == df['nominee'].max()].sort_values(by='index').head(1)['index'].values[0]

nominee_information[nominee_information['name'] == name]['top_genre']

## Top Actor Ratings by Genre `Google`

Find the top actors based on their average movie rating within the genre they appear in most frequently.

-  For each actor, determine their most frequent genre (i.e., the one they’ve appeared in the most).
-   If there is a tie in genre count, select the genre where the actor has the highest average rating.
-   If there is still a tie in both count and rating, include all tied genres for that actor.


Rank all resulting actor + genre pairs in descending order by their average movie rating.
-  Return all pairs that fall within the top 3 ranks (not simply the top 3 rows), including ties.
-  Do not skip rank numbers — for example, if two actors are tied at rank 1, the next rank is 2 (not 3).

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = top_actors_rating.groupby(['actor_name','genre'],as_index=False).agg(
    {
        'movie_title' : 'count',
        'movie_rating' : 'mean'
    })

max_count = df.groupby('actor_name',as_index=False)['movie_title'].max()

df = df.merge(max_count,on='actor_name')

df = df[df['movie_title_x'] == df['movie_title_y']]

max_rating = df.groupby('actor_name',as_index=False)['movie_rating'].max()

df = df.merge(max_rating,on='actor_name')

df = df[df['movie_rating_x'] == df['movie_rating_y']]

avg_rating = top_actors_rating.groupby(['actor_name','genre'],as_index=False)['movie_rating'].mean()

df = df.merge(avg_rating,on=['actor_name','genre']).sort_values(by='movie_rating',ascending=False)[['actor_name','genre','movie_rating']]

df['movie_rank'] = df['movie_rating'].rank(ascending=False,method='first')

df.query('movie_rank < 6').drop(columns='movie_rank')

## Monthly Percentage Difference `Amazon`

Given a table of purchases by date, calculate the month-over-month percentage change in revenue. The output should include the year-month date (YYYY-MM) and percentage change, rounded to the 2nd decimal point, and sorted from the beginning of the year to the end of the year.
The percentage change column will be populated from the 2nd month forward and can be calculated as ((this month's revenue - last month's revenue) / last month's revenue)*100.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
sf_transactions['month'] = sf_transactions['created_at'].dt.to_period('M')

df = sf_transactions.groupby(['month'],as_index=False)['value'].sum()

df['shifted'] = df['value'].shift(1)

df['revenue_diff_pct'] = ((df['value'] - df['shifted'])/df['shifted'])*100

df.drop(columns=['value','shifted'])

## Customer Tracking `Shopify`

Given the users' sessions logs on a particular day, calculate how many hours each user was active that day.


Note: The session starts when state=1 and ends when state=0.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
cust_tracking['shifted'] = cust_tracking.groupby('cust_id')['timestamp'].shift(1)
df = cust_tracking.query('state == 0')

df['delta'] = df['timestamp'] - df['shifted']

final = df.groupby(['cust_id'],as_index=False)['delta'].sum()

final['total_hours'] = final['delta']/3600

final[['cust_id','total_hours']]

## User Streaks `Linkedin`

Provided a table with user id and the dates they visited the platform, find the top 3 users with the longest continuous streak of visiting the platform as of August 10, 2022. Output the user ID and the length of the streak.


In case of a tie, display all users with the top three longest streaks.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = user_streaks.query("date_visited < '2022-08-11'").sort_values(by=['user_id','date_visited']).drop_duplicates()

df['prev'] = df.groupby('user_id')['date_visited'].shift(1)

df.dropna(inplace=True)

df['delta'] = ((df['date_visited'] - df['prev'])/3600)/24

df['delta'] = df['delta'].dt.seconds

def streak(uid):
    values = df[df['user_id'] == uid]['delta'].values
    stk = 1
    for i in values:
        if i == 1:
            stk += 1
        else:
            stk = 1
    return stk

final = pd.DataFrame(columns=['user_id','streak_length'])

l = []
for i in df['user_id'].unique():
    l.append(streak(i))

final['user_id'] = df['user_id'].unique()
final['streak_length'] = l

final = final.sort_values(by='streak_length',ascending=False)

final['ranks'] = final['streak_length'].rank(ascending=False,method='dense')
final.query('ranks < 4')[['user_id','streak_length']]

## Marketing Campaign Success [Advanced] `ActiveCampaign`

You have the marketing_campaign table, which records in-app purchases by users. Users making their first in-app purchase enter a marketing campaign, where they see call-to-actions for more purchases. Find how many users made additional purchases due to the campaign's success.


The campaign starts one day after the first purchase. Users with only one or multiple purchases on the first day do not count, nor do users who later buy only the same products from their first day.

In [ ]:
# Import your libraries
import pandas as pd
import numpy as np

count = 0
for i in marketing_campaign['user_id'].unique():
    df = marketing_campaign.query(f'user_id == {i}')
    product = df[df['created_at'] == df['created_at'].min()]['product_id'].values
    camp = df[df['created_at'] != df['created_at'].min()]
    val = np.setdiff1d(camp['product_id'].unique(),product)
    if len(val) > 0:
        count += 1

pd.DataFrame([count],columns=['user_count'])

## Reviewed flags of top videos `Google`

For the video (or videos) that received the most user flags, how many of these flags were reviewed by YouTube? Output the video ID and the corresponding number of reviewed flags.


In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = user_flags.merge(flag_review,on='flag_id')

temp = df['video_id'].value_counts().reset_index()

df[df['video_id'].isin(temp[temp['video_id'] == temp['video_id'].max()]['index'].values)].groupby('video_id')['reviewed_by_yt'].sum().reset_index()

## Top 5 States With 5 Star Businesses `Yelp`

Find the top 5 states with the most 5 star businesses. Output the state name along with the number of 5-star businesses and order records by the number of 5-star businesses in descending order. In case there are ties in the number of businesses, return all the unique states. If two states have the same result, sort them in alphabetical order.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
df = yelp_business.query('stars == 5')['state'].value_counts().reset_index()
df['rank'] = df['state'].rank(ascending=False,method='dense')
df.query('rank < 6')[['index','state']]

## City With Most Amenities `Airbnb`

You're given a dataset of searches for properties on Airbnb. For simplicity, let's say that each search result (i.e., each row) represents a unique host. Find the city with the most amenities across all their host's properties. Output the name of the city.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
airbnb_search_details['amenities'] = airbnb_search_details['amenities'].str[1:-1].str.replace('"','').str.lower().str.strip().str.split(',')

df = airbnb_search_details.groupby('city',as_index=False)['amenities'].sum()

df['l'] = df['amenities'].str.len()

df.sort_values(by='l',ascending=False)['city'].head(1)

## Popularity Percentage `Meta`

Find the popularity percentage for each user on Meta/Facebook. The dataset contains two columns, user1 and user2, which represent pairs of friends. Each row indicates a mutual friendship between user1 and user2, meaning both users are friends with each other. A user's popularity percentage is calculated as the total number of friends they have (counting connections from both user1 and user2 columns) divided by the total number of unique users on the platform. Multiply this value by 100 to express it as a percentage.


Output each user along with their calculated popularity percentage. The results should be ordered by user ID in ascending order.

In [ ]:
# Import your libraries
import pandas as pd
import numpy as np

# Start writing code
x= facebook_friends['user1'].value_counts().reset_index()
y= facebook_friends['user2'].value_counts().reset_index()

df = x.merge(y,left_on='user1',how='outer',right_on='user2')

total = len(np.union1d(facebook_friends['user1'].unique(),facebook_friends['user2'].unique()))

df['user'] = df.apply(lambda x : x['user1'] if not pd.isnull(x['user1']) else x['user2'], axis=1)

df = df.fillna(0)

df['friends'] = df['count_x'] + df['count_y']

df['popularity_percent'] = (df['friends']/total)*100

df[['user','popularity_percent']]

## Top Percentile Fraud `Google`

We want to identify the most suspicious claims in each state. We'll consider the top 5 percentile of claims with the highest fraud scores in each state as potentially fraudulent.


Your output should include the policy number, state, claim cost, and fraud score.

In [ ]:
# Import your libraries
import pandas as pd
import numpy as np

# Start writing code
df = fraud_score.sort_values(by=['state','fraud_score'],ascending=[True,False])

per = pd.DataFrame(columns=['state','percentile'])

l = []

for i,j in df.groupby('state',as_index=False):
    l.append([i,j['fraud_score'].quantile(0.95)])

per[['state','percentile']] = l

df.merge(per,on='state').query('fraud_score >= percentile').drop(columns='percentile')


## Cookbook Recipes `Ebay`

You are given a table containing recipe titles and their corresponding page numbers from a cookbook. Your task is to format the data to represent how recipes are distributed across double-page spreads in the book.


Each spread consists of two pages:


-   The left page (even-numbered) and its corresponding recipe title (if any).
-   The right page (odd-numbered) and its corresponding recipe title (if any).


The output table should contain the following three columns:


-   left_page_number – The even-numbered page that starts each double-page spread.
-   left_title – The title of the recipe on the left page (if available).
-   right_title – The title of the recipe on the right page (if available).


For the  k-th  row (starting from 0):


-   The  left_page_number  should be $2 * k$.
-   The  left_title  should be the title from page $2 * k$, or NULL if there is no recipe on that page.
-   The  right_title  should be the title from page $2 * k + 1$, or NULL if there is no recipe on that page.


Each page contains at most one recipe and  if a page does not contain a recipe, the corresponding title should be NULL. Page 0 (the inside cover) is always empty and included in the output. The table should ensure that all pages up to the maximum recorded page number are included, even if they contain

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code

l = [i for i in range(0,cookbook_titles.shape[0]*2,2)]
df = pd.DataFrame(l,columns=['page_number'])

odd = cookbook_titles[cookbook_titles['page_number']%2 != 0]
odd['page_number'] = odd['page_number'] - 1

even = cookbook_titles[cookbook_titles['page_number']%2 == 0]

df.merge(even,on='page_number',how='left').merge(odd,on='page_number',how='left')

## Retention Rate `Meta`

You are given a dataset that tracks user activity. The dataset includes information about the date of user activity, the account_id associated with the activity, and the user_id of the user performing the activity. Each row in the dataset represents a user’s activity on a specific date for a particular account_id.


Your task is to calculate the monthly retention rate for users for each account_id for December 2020 and January 2021. The retention rate is defined as the percentage of users active in a given month who have activity in any future month.


For instance, a user is considered retained for December 2020 if they have activity in December 2020 and any subsequent month (e.g., January 2021 or later). Similarly, a user is retained for January 2021 if they have activity in January 2021 and any later month (e.g., February 2021 or later).


The final output should include the account_id and the ratio of the retention rate in January 2021 to the retention rate in December 2020 for each account_id. If there are no users retained in December 2020, the retention rate ratio should be set to 0.

In [ ]:
import pandas as pd

df12 = sf_events[sf_events['record_date'].dt.month == 12]

df1 = sf_events[sf_events['record_date'].dt.month == 1]

df_dec = sf_events.query('record_date > "2020-12-31"').merge(df12,on=['account_id','user_id'])

df_jan = sf_events.query('record_date > "2021-01-31"').merge(df1,on=['account_id','user_id'])

dec_retend = df_dec.groupby(['account_id'])['user_id'].nunique() / df12.groupby(['account_id'])['user_id'].nunique()

jan_retend = df_jan.groupby(['account_id'])['user_id'].nunique() / df1.groupby(['account_id'])['user_id'].nunique()

jan_retend.fillna(0,inplace=True)

(jan_retend/dec_retend).reset_index()

## Counting Instances in Text `Google`

Find the number of times the exact words bull and bear appear in the contents column.


Count all occurrences, even if they appear multiple times within the same row. Matches should be case-insensitive and only count exact words, that is, exclude substrings like bullish or bearing.


Output the word (bull or bear) and the corresponding number of occurrences.

In [ ]:
# Import your libraries
import pandas as pd

# Start writing code
google_file_store['contents'] = google_file_store['contents'].str.lower().str.split(' ')
bull = google_file_store['contents'].apply(lambda x : sum([True if i == 'bull' else False for i in x])).sum()

bear = google_file_store['contents'].apply(lambda x : sum([True if i == 'bear' else False for i in x])).sum()

pd.DataFrame([['bull',bull],['bear',bear]],columns=['word','netry'])

## Most Popular Client For Calls `Apple`

Select the most popular client_id based on the number of users who individually have at least 50% of their events from the following list: 'video call received', 'video call sent', 'voice call received', 'voice call sent'.

In [ ]:
# Import your libraries
import pandas as pd
import numpy as np

# Start writing code
l = ['video call received', 'video call sent', 'voice call received', 'voice call sent']
df = fact_events.groupby('user_id',as_index=False).agg(
    {'event_type' : set}
    )

df['event_type'] = df['event_type'].apply(lambda x : list(x))

df['common'] = df.apply(lambda x : np.intersect1d(x['event_type'],l),axis=1)

df['fifty'] = df['common'].apply(len)/df['event_type'].apply(len)

df = df.query('fifty >= 0.5')

df.merge(fact_events,on='user_id').groupby('client_id',as_index=False)['user_id'].nunique()['client_id']